
# **Principal Strains Prediction Using Inertial Measurement Units**

**By Dr. Zainab Altai**  
University of Essex  
z.altai@essex.ac.uk  
December 2024  

This Python script is designed for use on Google Colab. It predicts principal strains (e1 and e3) from inertial measurement unit (IMU) data using a pre-trained machine learning model (XCM). The script performs the following tasks:

1. Install necessary libraries.  
2. Import and preprocess data from MATLAB files.  
3. Split data into training and testing sets.  
4. Load the pre-trained XCM model.  
5. Predict and evaluate principal strains.  

---

### Instructions to Use the Code
1. Clone the repository or copy this script to your Google Colab notebook.
2. Update file paths to match your Google Drive directories.
3. Run each cell in order.
4. Modify the `output_var` to switch between predicting `e1` and `e3`.
    

#### 1. Install Necessary Libraries

In [ ]:

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Install required libraries
!pip install mat4py
!pip install -Uqq tsai

# Import libraries
from tsai.all import *
import numpy as np
from mat4py import loadmat
import h5py


#### 2. Import and Preprocess Data

In [ ]:

# Load MATLAB file (update directory as needed)
data = loadmat('/content/drive/MyDrive/ColabNotebooks/AMS_ML/InputData/IMU_FE/DataCombined_IMU_FE_allTasks_ML_data_double_RTH_LTH_PEL.mat')
output = data["outputs"]
input = data["inputs"]

# Load labels (update directory as needed)
df = pd.read_csv(r'/content/drive/MyDrive/ColabNotebooks/AMS_ML/InputData/IMU_FE/DataCombined_IMU_FE_allTasks_ML_labels.csv')

# Function to extract and restructure data
def extractor(data):
    all_outputs = np.dstack(list(data["outputs"].values()))
    x_axis_input = np.dstack(list(data["inputs"]['x'].values()))
    y_axis_input = np.dstack(list(data["inputs"]['y'].values()))
    z_axis_input = np.dstack(list(data["inputs"]['z'].values()))
    all_inputs = np.concatenate((x_axis_input, y_axis_input, z_axis_input), axis=-1)
    return all_outputs, all_inputs

# Extract inputs and outputs
all_outputs, all_inputs = extractor(data)
X2 = np.moveaxis(np.array(all_inputs), -1, 1)

# Define output variable (modify for e1 or e3)
output_var = 'e1'  # or 'e3'
Y2 = np.array(all_outputs[:, :, 0 if output_var == 'e1' else 1])


#### 3. Split Data into Training and Testing Sets

In [ ]:

from sklearn.model_selection import StratifiedShuffleSplit

# Ensure labels match data
tasks = df['Task'].values
assert len(df) == len(X2), "Label length mismatch."

# Stratified split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in sss.split(np.zeros(len(tasks)), tasks):
    X_train, X_test = X2[train_index], X2[test_index]
    y_train, y_test = Y2[train_index], Y2[test_index]

# Save indices and data (update output_dir)
output_dir = "/content/drive/MyDrive/ColabNotebooks/AMS_ML/OutputData/TypicalSplits/XCM/IMU_FE/allTasks_threeIMU/"
train_indices_df = pd.DataFrame(train_index, columns=['Index'])
test_indices_df = pd.DataFrame(test_index, columns=['Index'])
train_indices_df.to_csv(f'{output_dir}train_indices_{output_var}.csv', index=False)
test_indices_df.to_csv(f'{output_dir}test_indices_{output_var}.csv', index=False)


#### 4. Load Pre-Trained Model

In [ ]:

from tensorflow import keras

# Load trained model (update path as needed)
model_path = f"/content/drive/MyDrive/ML/Outputs/SubjectSplits/XCM/model_{output_var}_XCM.pk1"
learn = load_learner(model_path, cpu=False)


#### 5. Predict and Evaluate Principal Strains

In [ ]:

# Get predictions
probas, _, preds = learn.get_X_preds(X_test)  # X_test = X[splits[1]]
predictions = np.array(preds)
target = np.array(y_test)

# Save predictions and targets
np.savetxt(f"{output_dir}predictions_{output_var}_XCM.csv", predictions.T, delimiter=',')
np.savetxt(f"{output_dir}originals_{output_var}_XCM.csv", target.T, delimiter=',')

# Evaluation metrics
def RMSE(target, predictions):
    return np.sqrt(np.mean((target - predictions) ** 2, axis=1))

def relRMSE(target, predictions):
    nom = RMSE(target, predictions)
    denom = 0.5 * (np.ptp(target, axis=1) + np.ptp(predictions, axis=1))
    return (nom / denom) * 100

def cor_fun(target, predictions):
    return np.array([np.corrcoef(target[i], predictions[i])[0, 1] for i in range(target.shape[0])])

results = {
    "RMSE": RMSE(target, predictions),
    "relRMSE": relRMSE(target, predictions),
    "cor": cor_fun(target, predictions),
}

# Save metrics
np.savetxt(f"{output_dir}RMSE_{output_var}_XCM.txt", results['RMSE'], fmt='%f')
np.savetxt(f"{output_dir}relRMSE_{output_var}_XCM.txt", results['relRMSE'], fmt='%f')
np.savetxt(f"{output_dir}cor_{output_var}_XCM.txt", results['cor'], fmt='%f')
